# Lab Two: Classification #

Billy Nayden
Sean McWhirter
Andrew Mejia
Rajesh Salturi

In [ ]:
import plotly
plotly.offline.init_notebook_mode()
import pandas as pd
import numpy as np 
import seaborn as sns 
from mpl_toolkits.mplot3d import Axes3D 
import matplotlib.pyplot as plt
import time
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.svm import SVC
from pandas.plotting import boxplot
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import learning_curve
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error, make_scorer, mean_squared_error, mean_squared_error
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.kernel_approximation import Nystroem
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import MinMaxScaler
from yellowbrick.model_selection import FeatureImportances

print(__doc__)


##### CLASSIFICATION ANALYSIS HELPER FUNCTIONS

In [ ]:
# Adapted from:
# https://www.featureranking.com/tutorials/machine-learning-tutorials/information-gain-computation/
#https://nbviewer.jupyter.org/github/jakemdrew/MachineLearningExtras/blob/master/LFW%20Dataset%20and%20Class%20Imbalance.ipynb

def gini_index(y):
    probs = pd.value_counts(y,normalize=True)
    return 1 - np.sum(np.square(probs))

def plot_class_dist(y, target_label = None):
    fig, axarr = plt.subplots(1, 2, figsize=(18, 6))
    class_ct = len(np.unique(y))
    vc = pd.value_counts(y)
    print('Total Records', len(y))
    print('Total Classes:', class_ct)
    print('Class Gini Index', gini_index(y))
    print('Smallest Class Id:',vc.idxmin(),'Records:',vc.min())
    print('Largest Class Id:',vc.idxmax(),'Records:',vc.max())
    print('Accuracy when Guessing:', np.round( (1 / len(np.unique(y))) * 100, 2), '%')

    sns.distplot(y, ax=axarr[0], bins=class_ct).set_title('Target Class Distribution:', target_label);
    sns.distplot(y, ax=axarr[1], kde=False, bins=class_ct).set_title('Target Class Counts:', target_label);
    
    
# This function creates dummy encodings from a lsit of features of interest and returns a dataframe     
def create_dummy_encod(ml_df,features_of_interest, drop_first_cat=True, sparsity=True): 
    tmp_cont = []
    ml_data_copy = ml_df.copy()
    for feat in features_of_interest: 
        tmp_df = pd.get_dummies(ml_data_copy[feat],prefix=str(feat),sparse=sparsity,drop_first=drop_first_cat)
        tmp_cont.append(tmp_df)
        feat_df = pd.concat(tmp_cont,axis=1)
        ml_df = pd.concat([ml_data_copy,feat_df], axis=1)
        ml_df = ml_df.drop(columns = features_of_interest, axis = 1)
    return ml_df 



#Adopted from 
#https://nbviewer.jupyter.org/github/jakemdrew/MachineLearningExtras/blob/master/LFW%20Dataset%20and%20Class%20Imbalance.ipynb

# This function provides a way to use stratiefied cv with the test models function below using the default score of accuarcy 

cv = StratifiedKFold(n_splits=10,shuffle=True, random_state=959)

def stratified_cross_validate(model, X, y, cv=cv):
    start = time.time()
    cv_results = cross_validate(model, X, y, cv=cv, scoring="accuracy", n_jobs=-1)
    elapsed_time = (time.time() - start) 
    print ('Fold Scores:')
    print(' ')
    print(cv_results['test_score'])
    print(' ')
    print('Mean Accuracy: ', cv_results['test_score'].mean())
    print('Mean Fit Time: ', cv_results['fit_time'].mean())
    print('Mean Score Time: ', cv_results['score_time'].mean())
    print('CV Time: ', elapsed_time)
    return


#This function provides a quick method to to performe strativied cross validation model comparision 

def test_models(X, y, models, model_names):
    for model, model_name in zip(models,model_names):
        print(model_name)
        print('--------------------------------')
        stratified_cross_validate(model,X,y)
        print(' ')

#This function will plot PCs based on the length of features in the dataframe or change to how many features you wish to input 
def plot_pca(X,var_ratio_pcs = 60):
    # Perform PCA on the data to reduce the number of initial features 
    # and to remove correlations that are common between pixel features 
    pca = PCA(n_components=X.shape[1])
    pca.fit(X)

    # Inspect the explained variances to determine how many components to use  
    plt.subplots(figsize=(8, 8))
    plt.plot(np.cumsum(pca.explained_variance_ratio_))
    plt.xlabel('number of components')
    plt.ylabel('cumulative explained variance');
    print('Cumulative Explained variance explained with :', var_ratio_pcs , 'components:',sum(pca.explained_variance_ratio_[0:var_ratio_pcs]) )

#### Regression Analysis Helper Functions

In [ ]:
#Adopted from https://github.com/jakemdrew/DataMiningNotebooks/blob/master/07.%20Regression.ipynb

#Function for Root mean squared error
#https://stackoverflow.com/questions/17197492/root-mean-square-error-in-python
def rmse(y_actual, y_predicted):
    return np.sqrt(mean_squared_error(y_actual, y_predicted))

#Function for Mean Absolute Percentage Error (MAPE) - Untested
#Adapted from - https://stackoverflow.com/questions/42250958/how-to-optimize-mape-code-in-python
def mape(y_actual, y_predicted): 
    mask = y_actual != 0
    return (np.fabs(y_actual - y_predicted)/y_actual)[mask].mean() * 100

#Create scorers for rmse and mape functions
mae_scorer = make_scorer(score_func=mean_absolute_error, greater_is_better=False)
rmse_scorer = make_scorer(score_func=rmse, greater_is_better=False)
mape_scorer = make_scorer(score_func=mape, greater_is_better=False)
mse_scorer = make_scorer(score_func = mean_squared_error, greater_is_better=False)

#Make scorer array to pass into cross_validate() function for producing mutiple scores for each cv fold.
errorScoring = {'MAE':  mae_scorer, 
                'RMSE': rmse_scorer,
                'MAPE': mape_scorer, 
                'MSE' : mse_scorer
               } 

def EvaluateRegressionEstimator(regEstimator, X, y, cv):
    
    scores = cross_validate(regEstimator, X, y, scoring=errorScoring, cv=cv, return_train_score=True)

    #cross val score sign-flips the outputs of MAE
    # https://github.com/scikit-learn/scikit-learn/issues/2439
    scores['test_MAE'] = scores['test_MAE'] * -1
    scores['test_MAPE'] = scores['test_MAPE'] * -1
    scores['test_RMSE'] = scores['test_RMSE'] * -1
    scores['test_MSE'] = scores['test_MSE'] * -1

    #print mean MAE for all folds 
    maeAvg = scores['test_MAE'].mean()
    print_str = "The average MAE for all cv folds is: \t\t\t {maeAvg:.5}"
    print(print_str.format(maeAvg=maeAvg))

    #print mean test_MAPE for all folds
    scores['test_MAPE'] = scores['test_MAPE']
    mape_avg = scores['test_MAPE'].mean()
    print_str = "The average MAE percentage (MAPE) for all cv folds is: \t {mape_avg:.5}"
    print(print_str.format(mape_avg=mape_avg))

    #print mean MAE for all folds 
    RMSEavg = scores['test_RMSE'].mean()
    print_str = "The average RMSE for all cv folds is: \t\t\t {RMSEavg:.5}"
    print(print_str.format(RMSEavg=RMSEavg))
    print('*********************************************************')
    
    #print mean MAE for all folds 
    MSEavg = scores['test_MSE'].mean()
    print_str = "The average MSE for all cv folds is: \t\t\t {MSEavg:.5}"
    print(print_str.format(MSEavg=MSEavg))
    print('*********************************************************')

    print('Cross Validation Fold Mean Error Scores')
    scoresResults = pd.DataFrame()
    scoresResults['MAE'] = scores['test_MAE']
    scoresResults['MAPE'] = scores['test_MAPE']
    scoresResults['RMSE'] = scores['test_RMSE']
    scoresResults['MSE'] = scores['test_MSE']
    return scoresResults

class CappedLinearRegression(LinearRegression):

    def predict(self, X):
        return np.clip(super(CappedLinearRegression, self).predict(X), 1e5, 9.9e6) 

#### Other Helper Functions

In [ ]:
def remove_white_space(cols_list, dataframe): 
    df = dataframe
    for col in cols_list:
        df[col] = df[col].str.strip()
    return df 
    
    
def unique_categories(columns_list, dataframe_1): 
    miss_cat_vars = {}
    for var in columns_list: 
        print(var)
        k,v = var,dataframe_1[var].unique()
        miss_cat_vars.update({k : v})
    return miss_cat_vars

#Adapted from Jezreal's answer 
#https://stackoverflow.com/questions/51189962/how-to-replace-0-values-with-mean-based-on-groupby

def grouper_impute(dataframe_2, grouper_col = None, grouper_impute = None, replace_val = None, transfrmtn = None): 

    dataframe_2[grouper_impute] = dataframe_2[grouper_impute].replace(replace_val, np.nan)
    dataframe_2[grouper_impute] = dataframe_2[grouper_impute].fillna(dataframe_2.groupby(grouper_col)[grouper_impute].transform(transfrmtn))
    return dataframe_2
        



# Data Preparation Part 1
>Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis.*

# Define and prepare your class variables.

>The data set we will be working with is the NYC property data sales from New York City for the 5 boroghs.

>https://www.kaggle.com/new-york-city/nyc-property-sales

>We wanted to create a classifier using this data set to predict which BOROUGH a property belonged to, and we wanted to create a regressor to predict a property's SALE PRICE.

This dataset contains the location, address, type, sale price, and sale date of building units sold. A reference on the trickier fields:

BOROUGH: A digit code for the borough the property is located in; in order these are Manhattan (1), Bronx (2), Brooklyn (3), Queens (4), and Staten Island (5).

BLOCK; LOT: The combination of borough, block, and lot forms a unique key for property in New York City. Commonly called a BBL. We will not use this combination in our analysis, as we will treat Block and Lot as independent features, since it is a nominal label created using this type of key value pair.

BUILDING CLASS AT PRESENT and BUILDING CLASS AT TIME OF SALE: The type of building at various points in time. See the glossary linked to below.

Note that because this is a financial transaction dataset, there are some points that need to be kept in mind:

There are sales with a $0 dollar value. These sales are actually transfers of deeds between parties: for example, parents transferring ownership to their home to a child after moving out for retirement.

In [ ]:
raw_data_url = 'https://raw.githubusercontent.com/andrewmejia600/MSDS7331/master/RAW_DATA/nyc-rolling-sales.csv'

In [ ]:
raw_data = pd.read_csv(raw_data_url, encoding="utf-8", converters = {'LAND SQUARE FEET': str.strip, 'GROSS SQUARE FEET' : str.strip, 'SALE PRICE': str.strip  } )
raw_data.head(n=5)

In [ ]:
raw_data.columns

The raw data attributes explainations are as follows: 

Unnamed: 0 - An index from the data source that is not defined 

BOROUGH - A digit code for the borough the property is located in; in order these are Manhattan (1), Bronx (2), Brooklyn (3), Queens (4), and Staten Island (5). 

NEIGHBOHOOD - The neighboorhood of the address 

BUILDING CLASS CATEGORY - Is the description of the building class, reference https://www1.nyc.gov/assets/finance/jump/hlpbldgcode.html 

TAX CLASS AT PRESENT - The Tax class of the building at the time when the data was collected 

BLOCK - The physical block of the address 

LOT - The lot on the the block 

EASE-MENT - A sparse column with no values in it, just white space. 

BUILDING CLASS AT PRESENT - The building class at the time of data collection 

ADDRESS - The Physical address of the building 

APARTMENT NUMBER - The Apartment Number of the unit 

ZIP CODE - Zip Code of the property

RESIDENTIAL UNITS - The number of residential units to the building 

COMMERCIAL UNITS - The number of commerifal units to the building 

Total Units - The number of total units, the sum of Residential and Commercial Units to the building 

LAND SQUARE FEET - The physical foot print of square feet of the building 

GROSS SQUARE FEET -- The entire square footage of the buidling, the sum of the floor square footage for mutli-story buildings 

YEAR BUILT - The year of construction of the property 

TAX CLASS AT TIME OF SALE - The tax class of the building when the property sold 

Building CLASS AT TIME OF SALE - The building class at the time when the property sold 

SALE PRICE - The sale price of the building, $0 are indicative of family property transfers and '-' are where there is no known sale price 

SALE DATE - The sale date of the property

In [ ]:
raw_data.info()

In [ ]:
housing_data = raw_data.copy()


We see there are potentially 22 features for us to use for both the classification task of the data and the regression task of the data. We will drop the Unamed 0: column as it appears to be an index that is not of importance. 

EASE-MENT is another feature that is mostly sparse, and shows no real value as a feature. 

Address has many nominal values and will not serve as a sound predictor, and will drastically increase the number of features to be encoded, so we will drop this column as well.  

Therefore the further analysis and model testing will be conducted without Unamed:0, EASE-MENT, and Address. We will also replace SALE DATE with DATEOFSALE as DATEOFSALE will be used for downline processing and will be in date time format. 

In [ ]:
housing_data['DATEOFSALE'] = pd.to_datetime(housing_data['SALE DATE'])

In [ ]:
housing_data = housing_data.drop(columns = ['Unnamed: 0', 'EASE-MENT', 'APARTMENT NUMBER', 'SALE DATE'], axis = 1)

We are removing the whitespace from the column names to make them easier to manipulate in our modeling analysis and building.

In [ ]:
housing_data.columns = housing_data.columns.str.replace(' ', '')

In [ ]:
catagorical_vars = list(housing_data.select_dtypes(include='object').columns)
print(catagorical_vars)

In [ ]:
housing_data = remove_white_space(catagorical_vars, housing_data)

In [ ]:
housing_data.info()

As we are not sure what the '-' value for sale price means, we will omit it from further analysis, as these could be family property transfers or transfers other than sales where there is not a $0 value or there is no other data known for this observation. We see there are 14561 observations of this type. It must be stated, for future generalizations, the outcomes of these regression and classification models are limited to observations that do not fall into this category. 

In [ ]:
len(housing_data[housing_data['SALEPRICE'] == '-']) 

We see this is also the case with LANDSQUAREFEET and GROSSSQUAREFEET as well and we see there are instances where there is a 0 zipcode. As there is no way to vet what the true zipcode of the property would be, without extensive searching, we will exclude these observations as well. We also see there are some properties with a 0 total units, so we will exclude these observations, as we do not want to artifically inflate the number of total units by adding to the column. With these exclusions, we must restate our previous disclaimer of the outcomes of these regression and classification models are limited to observations that do not fall into this category.

In [ ]:
housing_data = housing_data[~((housing_data['LANDSQUAREFEET'] == '-') | (housing_data['GROSSSQUAREFEET'] == '-') | (housing_data['SALEPRICE'] == '-'))]

In [ ]:
housing_data = housing_data[(housing_data['ZIPCODE'] != 0) & (housing_data['TOTALUNITS'] != 0) ] 

In [ ]:
housing_data = housing_data.reset_index(drop=True)

In [ ]:
housing_data.info()

We see some attributes, such as LANDSQUREFEET, GROSSSUAREFEET and SALEPRICE are the wrong data type. 
We are changing the data types to SALEPRICE, LANDSQUAREFEET and GROSSSQUAREFEET to int64, and making ZIPCODE a nominal label, as ZIPCODE is more of a nominal label in this case rather than an ordinal value, since a higher ZIPCODE number does not necessiarily correspond to a more favorable property.

In [ ]:
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html
housing_data = housing_data.astype({'SALEPRICE': 'int64', 'LANDSQUAREFEET' : 'int64', 'GROSSSQUAREFEET' : 'int64', 'ZIPCODE' : 'object'})

## ENGINEERED FEATURES AND IMPUTATIONS

We are creating an ordinal feature for when in the month the property sold, 1 being the first of the month, 2 being the middle of the month and 3 being the last of the month. 

In [ ]:
string_of_day = housing_data.DATEOFSALE.astype(str).str.slice(start=-2).astype(int)

housing_data['TIMEOFMONTH'] = pd.cut(string_of_day, [1,14,15,31], labels=[1,2,3], include_lowest=True)

In [ ]:
housing_data.TIMEOFMONTH = housing_data.TIMEOFMONTH.astype(object)

When reviewing https://www1.nyc.gov/assets/finance/jump/hlpbldgcode.html the New York Building class at present feature has many nominal features. We wanted a more general feature of what the building type is, i.e. whether the property is a multi family dwelling with vs  with out an elevator. 


>'DWELLING' -- DWELLING 
'MULTI_DWELLING_NO_ELV'-- Multifamily dwelling no elevator 
'MULTI_DWELLING_ELV' -- Multifamily dwelling with elevator
'WAREHOUSE' -- Warehouse
'FACTORY' -- Factory
'GARAGE_P_LOT' -- Garage or Parking Lot
'HOTEL' -- Hotel 
'HOSIPTAL' -- Hospital 
'THEATRE' -- Theatre
'RETAIL' -- Retail 
'LOFT' -- Loft 
'RELIGIOUS' -- Religious 
'SOCIAL_INSTITUTION' -- Social Institution, i.e. an Asylum 
'OFFICE' -- Office 
'COMMUNITY_CENTER' -- Community Center
'PUBLIC_REC' -- Public Recreation
'COMMERCIAL' -- Commercial 
'DWELLING_RETAIL' -- Dwelling with retail 
'PORT' -- Port of entry 
'UTILITY' -- Utility, i.e. railroad 
'ZONED' -- Zoned, i.e. Courthouse
'SCHOOL' -- Schools 
'PUBLIC_SAFETY' -- Public Safety, i.e. Firehouse 
'OTHER' -- Catchall 

In [ ]:
#https://www1.nyc.gov/assets/finance/jump/hlpbldgcode.html#A
BUILD_CLASS_DICT_ = {'A' : 'DWELLING', 'B' : 'DWELLING', 'C' : 'MULTI_DWELLING_NO_ELV', 'D' : 'MULTI_DWELLING_ELV', 'E' : 'WAREHOUSE', 'F' : 'FACTORY', 'G' : 'GARAGE_P_LOT', 'H' : 'HOTEL', 'I' : 'HOSIPTAL', 'J' : 'THEATRE', 'K' : 'REATAIL', 'L' : 'LOFT', 'M' : 'RELIGIOUS', 'N' : 'SOCIAL_INSTITUTION', 'O' : 'OFFICE', 'P' : 'COMMUNITY_CENTER', 'Q' : 'PUBLIC_REC', 'R' : 'COMMERCIAL' , 'S' : 'DWELLING_RETAIL', 'T' : 'PORT', 'U' : 'UTILITY', 'V' : 'ZONED', 'W' : 'SCHOOL', 'Y' : 'PUBLIC_SAFETY', 'Z' : 'OTHER'}

housing_data['BUILDCLASSGENER'] = housing_data.BUILDINGCLASSATPRESENT.astype(str).str.slice(start=0, stop=1).map(BUILD_CLASS_DICT_)



We wanted to create a copy of the sale price for use with our imputations for the regression of sale price. 

In [ ]:
housing_data['SALEPRICECPY'] = housing_data['SALEPRICE']

We shall impute the $0 SALEPRICE feature, that are family transfers, with the median saleprice of the ZIPCODE. As we are assuming these values should be relatively close to each other if they are within the zipcode, since using Borogh could be too general. 

We will impute YEARBUILT, LANDSQUAREFEET and GROSSSQUREFEET with the median of the zipcode as well for similar reasons. 

In [ ]:
housing_data = grouper_impute(dataframe_2 = housing_data, grouper_col = 'ZIPCODE', grouper_impute = 'SALEPRICE', replace_val = 0, transfrmtn = 'median')

housing_data = grouper_impute(dataframe_2 = housing_data, grouper_col = 'ZIPCODE', grouper_impute = 'YEARBUILT', replace_val = 0, transfrmtn = 'median')


housing_data = grouper_impute(dataframe_2 = housing_data, grouper_col = 'ZIPCODE', grouper_impute = 'LANDSQUAREFEET', replace_val = 0, transfrmtn = 'median')

housing_data = grouper_impute(dataframe_2 = housing_data, grouper_col = 'ZIPCODE', grouper_impute = 'GROSSSQUAREFEET', replace_val = 0, transfrmtn = 'median')


We will engineer a feature for the decade of when the property was built. 

In [ ]:
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.pad.html
housing_data['DECADEBUILT'] = housing_data.YEARBUILT.astype(str).str.slice(start=0,stop=3).str.pad(width = 4, side='right', fillchar = str(0))

We will engineer a feature for the age of the property at time of sale, by subtracting the date of sale year from the year built. 

In [ ]:
housing_data['BUILDAGE'] = housing_data.DATEOFSALE.astype(str).str.slice(start=0,stop=4).astype(int) - housing_data.YEARBUILT

We will cast SALEPRICECPY, LANDSQUREFEET, GROSSSQUAREFEET, YEARBUILT, RESIDENTIALUNITS, BUILDAGE as int64 and cast ZIPCODE as a nomial feature. 

In [ ]:
housing_data = housing_data.astype({'SALEPRICECPY': 'int64', 'LANDSQUAREFEET' : 'int64', 
                                    'GROSSSQUAREFEET' : 'int64', 'ZIPCODE' : 'object',
                                   'YEARBUILT' : 'int64', 'RESIDENTIALUNITS' : 'int64'
                                   ,'BUILDAGE' : 'int64'})

In [ ]:
housing_data.info()

#### Accounting for duplicate records

We see there are 577 duplicate records, as we are not sure if these are errors, or are really different properties, we will omit these observations. 

In [ ]:
duplicate_records = housing_data[housing_data.duplicated(keep = False)]
duplicate_records

#### Removing Duplicate Records

In [ ]:
housing_data = housing_data[~housing_data.duplicated(keep = False)]

In [ ]:
housing_data.info()

At this time we will begin to formalize the approach for our experiment. 

We will create one dataset for the classification task, and one data set for the regression task. 

The reason for this, is as we performed some preliminary exploratory data analysis and model building, we saw the SALEPRICE attribute has a dynamic range of 0 to 2.2e9. As a result, prelminary regression models had a mean absolute error of ~1e6. There were many observations that were high leverage outliers that were driving the mean absolute error to ~1e6. It was decided this was not a practical method of predicting property values.  

Susbequent research should be done to consider contstructing models for regression of property values based on futher property value segmetation, i.e. 0 to 1e5, 1.01e5 to 1e6, 1.01e6 to 1.01e8. For this experiment, we will generalize our dynamic range of property values to be within 1e5 to 1e7, therefore any model findings are limited to property values within the range of 1e5 to 1e7. 

In [ ]:
housing_data.describe()

In [ ]:
cat_feat = list(housing_data.select_dtypes(include='object').columns)
cat_feat

# Data Preparation Part 2
>Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).*

# Classification Data

>The final data set for the classification task will include the following features: 

>NEIGHBORHOOD, BUILDINGCLASSCATEGORY , ZIPCODE, DECADEBUILT, TAXCLASSATPRESENT, 
BUILDINGCLASSATPRESENT, TIMEOFMONTH, BUILDCLASSGENER, RESIDENTIALUNITS, COMMERCIALUNITS, 
TOTALUNITS, LANDSQUAREFEET, GROSSSQUAREFEET, YEARBUILT, SALEPRICE 

>With BOROUGH as the target. 



# Regression Data 

Recall from our previous discussion, our Sale Price, in this case the SALEPRICECPY attribute will only have a dynamic range of 1e5 to 9.9e6, as to make our prediction error metric of mean absolute error more reasonable. 

>The final data set for the regression task will include the following features:


>NEIGHBORHOOD,BUILDINGCLASSCATEGORY, ZIPCODE, DECADEBUILT, TAXCLASSATPRESENT, 
BUILDINGCLASSATPRESENT, TIMEOFMONTH, BUILDCLASSGENER, RESIDENTIALUNITS, COMMERCIALUNITS, 
TOTALUNITS, LANDSQUAREFEET, GROSSSQUAREFEET, YEARBUILT, BUILDAGE, BOROUGH

>With SALEPRICECPY as the target

After conducting our imputations based by ZIPCODE and data exclusions, 
The final data attributes for both the regression and classification tasks explainations are as follows:

BOROUGH - A digit code for the borough the property is located in; in order these are Manhattan (1), Bronx (2), Brooklyn (3), Queens (4), and Staten Island (5).

NEIGHBOHOOD - The neighboorhood of the address

BUILDING CLASS CATEGORY - Is the description of the building class, reference https://www1.nyc.gov/assets/finance/jump/hlpbldgcode.html

TAXCLASSATPRESENT - The Tax class of the building at the time when the data was collected

BUILDINGCLASSATPRESENT - The building class at the time of data collection

ZIPCODE - Zip Code of the property

DECADEBUILT - Engineered feature for the decade of when the property was built 

TIMEOFMONTH - Engineered feature for when in the month the property was sold

BUILDCLASSGENER - Engineered feature of the building class categegory based on https://www1.nyc.gov/assets/finance/jump/hlpbldgcode.html

RESIDENTIAL UNITS - The number of residential units to the building

COMMERCIAL UNITS - The number of commerifal units to the building

Total Units - The number of total units, the sum of Residential and Commercial Units to the building

LAND SQUARE FEET - The physical foot print of square feet of the building

GROSS SQUARE FEET - The entire square footage of the buidling, the sum of the floor square footage for mutli-story buildings

YEARBUILT - The year of construction of the property

BUILDAGE - Engineered feature for regression only - the age of the building

SALEPRICE - Classification Only - The sale price of the building, 0 are indicative of family property transfers and '-' are where there is no known sale price and the $0 values were imputed with the zipecode median 

SALEPRICECPY - Regression Only - The engineered feature of SALEPRICE, but we impute the 0 value using the zipcode median only after we have we capped the property values from the data to be 1e5 to 9.9e6.

# CLASSIFICATION TASK DATA 

In [ ]:
housing_ml_df_classification = housing_data[['NEIGHBORHOOD','BUILDINGCLASSCATEGORY','ZIPCODE', 'DECADEBUILT', 'TAXCLASSATPRESENT', 
                              'BUILDINGCLASSATPRESENT', 'TIMEOFMONTH','BUILDCLASSGENER', 
                              'RESIDENTIALUNITS', 'COMMERCIALUNITS', 'TOTALUNITS',
                              'LANDSQUAREFEET', 'GROSSSQUAREFEET','YEARBUILT',
                              'SALEPRICE','BOROUGH']]

In [ ]:
housing_ml_df_classification.info()

# Regression TASK DATA 

In [ ]:
housing_data_regression = housing_data[(housing_data.SALEPRICECPY > 1e5) & (housing_data.SALEPRICECPY < 9.9e6)]

In [ ]:
housing_data_regression = housing_data_regression.drop(columns = ['SALEPRICE'], axis = 1)

In [ ]:
housing_data_regression = grouper_impute(dataframe_2 = housing_data_regression, grouper_col = 'ZIPCODE', grouper_impute = 'SALEPRICECPY', replace_val = 0, transfrmtn = 'median')

In [ ]:
housing_ml_df_regression = housing_data_regression[['NEIGHBORHOOD','BUILDINGCLASSCATEGORY','ZIPCODE', 'DECADEBUILT', 'TAXCLASSATPRESENT', 
                              'BUILDINGCLASSATPRESENT', 'TIMEOFMONTH','BUILDCLASSGENER', 
                              'RESIDENTIALUNITS', 'COMMERCIALUNITS', 'TOTALUNITS',
                              'LANDSQUAREFEET', 'GROSSSQUAREFEET','YEARBUILT',
                              'SALEPRICECPY','BUILDAGE','BOROUGH']]

In [ ]:
housing_ml_df_regression.info()

In [ ]:
#Corr plot
corr=housing_data.corr()
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(corr[(corr>= 0.1) | (corr <= -0.4)], cmap='viridis', vmax=1.0, vmin=-1.0, linewidths=0.1, annot=True, annot_kws={"size":11},ax=ax, square=True)

In [ ]:
#Extract all of our int64 variables and examine histograms
num_features = housing_data.loc[:, housing_data.dtypes == np.int64]
num_features.hist(figsize=(16, 20))

### THIS TAKES FOREVER TO RUN

Frequency plots for the cateorical Variables

In [ ]:
#Counts of the different categorical attributes
#Code derived from a Toward Data Science blog (link:https://towardsdatascience.com/how-to-perform-exploratory-data-analysis-with-seaborn-97e3413e841d)


#fig, ax=plt.subplots(3, 3, figsize=(20, 30))
#for attribute, subplot in zip(housing_data, ax.flatten()):
    #sns.countplot(housing_data[attribute], ax=subplot)
    #for label in subplot.get_xticklabels():
        #label.set_rotation(45)

Let's see how many classes we have and their frequencies 

In [ ]:
y_num_class = housing_data.BOROUGH.values

We see we have a highly imbalanced data set for Borough values for the outcomes and there is a 20% chance of guessing the class correctly. 
> Couldn't we get 47% just by ugessing 3 ever time?

In [ ]:
#Percentage of each borough -- if we just guessed 4 every time, what would our accuracy be?
housing_data['BOROUGH'].value_counts(normalize=True)*100

In [ ]:
#Is zipcode going to do the entire job for us, or is there some crossover?
sns.stripplot(x=housing_data['ZIPCODE'], y=housing_data['BOROUGH']);


In [ ]:
plot_class_dist(y_num_class) 

In [ ]:
feat_of_int = [ 'NEIGHBORHOOD', 'BUILDINGCLASSCATEGORY', 'ZIPCODE', 'DECADEBUILT','TAXCLASSATPRESENT', 'BUILDINGCLASSATPRESENT', 'TIMEOFMONTH','BUILDCLASSGENER']
housing_ml_df = housing_data[['NEIGHBORHOOD','BUILDINGCLASSCATEGORY','ZIPCODE', 'DECADEBUILT', 'TAXCLASSATPRESENT', 
                              'BUILDINGCLASSATPRESENT', 'TIMEOFMONTH','BUILDCLASSGENER', 
                              'RESIDENTIALUNITS', 'COMMERCIALUNITS', 'TOTALUNITS',
                              'LANDSQUAREFEET', 'GROSSSQUAREFEET','YEARBUILT',
                              'SALEPRICE','BOROUGH']]
ml_df_enc = create_dummy_encod(ml_df = housing_ml_df,features_of_interest = feat_of_int, drop_first_cat=True, sparsity=True)

In [ ]:
ml_df_enc.head(n=5)

# Modeling and Evaluation 1
>Choose and explain your evaluation metrics that you will use (i.e., accuracy,
precision, recall, F-measure, or any metric we have discussed). Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions.*

Given that we will be creating models for two different tasks, classification and regression, we will need to use two separate metrics to evaluate our models.

###### Classification
For our classification model, our sole concern is accuracy.  We need our model to be able to predict the correct borough with high accuracy in order for our model to be deployable.  Given that this is a multi-class classificaiton issue, our "guessing accuracy" is relatively low, so as long as our accuracy is high and our confusion matrix doesn't show any anomolies, we can be confident in the accuracy metric alone.  

###### Regression
In order to evaluate our regression model, we will want to use a meric that not only provides meaningful insight into the model's performance, but is also interpretable.  With these two parameters in mind, the metric we will use for our regression models is Mean Absolute Error (MAE).  It will allow us to conduct a meaningful evaluation, and being able to use the MAE value in terms of dollars (SALEPRICE being our regression target), making it interpretable. 

# Modeling and Evaluation 2
>Choose the method you will use for dividing your data into training and testing splits (i.e., are you using Stratified 10-fold cross validation? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. For example, if you are using time series data then you should be using continuous training and testing sets across time.*

###### Classification
This model uses 10-fold cross validation because our 70/30 train test split is subject to increased variance if we were to use a single holdout set. 10-fold cross validation helps reduce that variance.
###### Regression
To predict SALEPRICECPY, 10-fold cross valdiation will be used with a 80/20 train test split to account for the variance in the mean of the MAE of SALEPRICECPY. We will do this since the dynamic range of the property values are quite large, and we want to eliminate as much variance due to chance. 

# Modeling and Evaluation 3
>*Create three different classification/regression models for each task (e.g., random forest, KNN, and SVM for task one and the same or different algorithms for task two). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chosen metric. You must investigate different parameters of the algorithms!*

### Classification Section Sean/Billy Edits

In [ ]:
#Create scaler object 
ML_std_scalr = StandardScaler()


Let's see what our dataset gives us when we don't scale the data first

In [ ]:
y = ml_df_enc['BOROUGH'].values
X = ml_df_enc.drop(columns = ['BOROUGH'], axis = 1).values

In [ ]:
# Break our data into 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=959, stratify=y)

###    KNN Iteration 1 - No scaling/Stratified CV (10)

Let's see how KNN does without scaling our data first. We will use this as our baseline accuracy for this classification problem. The selection of 3 neighbors is arbitrary.

In [ ]:
#https://nbviewer.jupyter.org/github/jakemdrew/DataMiningNotebooks/blob/master/06.%20Classification.ipynb

cv = StratifiedKFold(n_splits=10,shuffle=True, random_state=959)
model = KNeighborsClassifier(n_neighbors=3, weights='uniform', metric='euclidean')
stratified_cross_validate(model, X, y, cv=cv)

It looks like about 60% without scaling our data and using an arbitrary K value.

Now let's loop through and find our optimal K to improve accuracy.

In [ ]:
cv = StratifiedKFold(n_splits=10,shuffle=True, random_state=959)
for K in range(1, 6):
    model = KNeighborsClassifier(n_neighbors=K, weights='uniform', metric='euclidean')
    print("K Value:", K)
    stratified_cross_validate(model, X, y, cv=cv)
    print("\n")
    print("----------------------------------")
    print("\n")

Based on `Mean Accuracy` 5 is the best K value for our unscaled model.

Now we will scale the data and re-run our loop to examine accuracy and identify the optimal K value.

###  KNN Iteration 2 - scaling/Stratified CV (10)

#### Warning: Takes about 5 mins to run

Scaling the data is necessary in this instance because KNN is a distance based model and will give higher weight to the variables with a larger scale. The below function brings the data into the same scale.

In [ ]:
#scale data
X = ML_std_scalr.fit_transform(X)

Now we will find the optimal K value for our model with scaled data.

In [ ]:
cv = StratifiedKFold(n_splits=10,shuffle=True, random_state=959)
for K in range(1, 6):
    model = KNeighborsClassifier(n_neighbors=K, weights='uniform', metric='euclidean')
    print("K Value:", K)
    stratified_cross_validate(model, X, y, cv=cv)
    print("\n")
    print("----------------------------------")
    print("\n")

Based on `Mean Accuracy`, 1 is the best K value for our scaled model with an accuracy of 99%.

In [ ]:
#Plot PCA 
plot_pca(X,var_ratio_pcs = 500)

It looks like it doesn't drastically reduce our dimensionality. It takes 500 components to explain 98.92% of the variance.  Unless we run into any major roadblocks with accuracy, we can just continue with scaled data.

# THIS TAKES A VERY LONG TIME--I STOPPED THE KERNEL AT 15 MINUTES

#PCA pipeline
from sklearn.pipeline import Pipeline

yhat = np.zeros(y.shape) # we will fill this with predictions

# create cross validation iterator
cv = StratifiedKFold(n_splits=10)

# setup pipeline to take PCA, then fit a KNN classifier
clf_pipe = Pipeline(
    [('PCA_Eric',PCA(n_components=500,svd_solver='randomized')),
     ('CLF_Eric',KNeighborsClassifier(n_neighbors=3))]
)

# now iterate through and get predictions, saved to the correct row in yhat
for train, test in cv.split(X,y):
    clf_pipe.fit(X[train],y[train])
    yhat[test] = clf_pipe.predict(X[test])

total_accuracy = mt.accuracy_score(y, yhat)
print ('KNN, pipeline accuracy', total_accuracy)

## Naive Bayes Iteration 1 - CV (10)

Since our dataset is sparse due to the OneHotEncoding, we will see whether Multinomial or Bernoulli is best.  The GausianNB will likely not be as effective due to the large number of zeros in the dataset.

The below code resets the data set to the original, unscaled values.

In [ ]:
y = ml_df_enc['BOROUGH'].values
X = ml_df_enc.drop(columns = ['BOROUGH'], axis = 1).values

Let's establish a baseline accuracy for the unscaled data, using the Multinomial Naive Bayes model. We will test across alpha values of 0.001, 0.5, and 1 to find the best alpha value. Additionally, we will use 10-fold cross validation to reduce the variance in our test set.

In [ ]:
#First, let's check multinomial
cv = StratifiedKFold(n_splits=10,shuffle=True, random_state=959)
for a in [0.001, 0.5, 1]:
    model= MultinomialNB(alpha=a)
    print("alpha value:", a)
    stratified_cross_validate(model, X, y, cv=cv)
    print("\n")
    print("------------------------")
    print("\n")

It appears that an alpha value of 0.001 is best for our Multinomial Naive Bayes model and produces an accuracy rating of 48.6%, far lower than our KNN model with unscaled data. 

Next, we will rerun the 10-fold cross validation with scaled data.  Due to the fact that StandardScalar can produce negative values, which does not work with Niave Bayese, we will use MinMaxScalar to function with the Niave Bayes models.

### Naive Bayes Iteration 2 - Scaled Data/Multinomial

In [ ]:



#Multinomial with scaled data
cv = StratifiedKFold(n_splits=10,shuffle=True, random_state=959)
#scale data using MinMaxScaler (Regular scaler provided negative numbers)
scaler = MinMaxScaler()
X = scaler.fit_transform(X)


#multinomialNB
for a in [0.001, 0.5, 1]:
    model= MultinomialNB(alpha=a)
    print("alpha value:", a)
    stratified_cross_validate(model, X, y, cv=cv)
    print("\n")
    print("------------------------")
    print("\n")

After iterating through various alpha values, it appears that MultinomialNB performs better with a lower alpha value (less smoothed data).  An alpha value of 0.001 produced a mean 10-fold cross-validated accuracy of 99.97%.

We will now test Bernoulli using a static alpha of 0.001, but testing binarize values of 0.001, 0.5, and 1. Again, we will use 10-fold cross validation.

## Naive Bayes Iteration 3 - Scaled Data/Bernoulli


In [ ]:

cv = StratifiedKFold(n_splits=10,shuffle=True, random_state=959)
#BernoulliNB
#We have already seen that we do not want a large alpha value for smoothing
#Let's iterate through the BernoulliNB with some different binarize values to see what produces the best results.
for b in [0.001, 0.5, 1]:
    model= BernoulliNB(alpha=0.001, binarize=b)
    print("binarize value:", b)
    stratified_cross_validate(model, X, y, cv=cv)
    print("\n")
    print("------------------------")
    print("\n")


Binarize values of 0.001 and 0.5 both produce very high accuracy values, but the binarize value of 0.001 performs slightly better in terms of time it takes to run, likely because it does not have to binarize more data.

Let's take a look at the confusion matrix to ensure that our high accuracy numbers are not disproportionately affected by a particular class.

With BernoulliNB and a binarize value of 0.001, we get an accuracy of 99.95%. 

### Logistic Regression Iteration 1 - Unscaled Data

Once again, we reset our data to the original, unscaled data set to use for our initial logistic regression model.

In [ ]:
#Reset Data after PCA
y = ml_df_enc['BOROUGH'].values
X = ml_df_enc.drop(columns = ['BOROUGH'], axis = 1).values

We will once again use 10-fold cross validation to determine a baseline accuracy score for logistic regression.

In [ ]:
#https://nbviewer.jupyter.org/github/jakemdrew/MachineLearningExtras/blob/master/LFW%20Dataset%20and%20Class%20Imbalance.ipynb
cv = StratifiedKFold(n_splits=10,shuffle=True, random_state=959)
model = LogisticRegression(solver='lbfgs', random_state=959, max_iter = 1e5)
stratified_cross_validate(model, X, y, cv=cv)

We will use 48.7% as our baseline accuracy for logistic regression. Let's take a look at the confusion matrix.

### Logistic Regression Iteration 2 - Scaled data

Once again, we will scale the features, in hopes of improving accuracy.

In [ ]:
#reset X and y
# Break our data into 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=959, stratify=y)

In [ ]:
#Scaling the features will likely make a big improvement to accuaracy 
X = ML_std_scalr.fit_transform(X)

Next, we will run the model using 10-fold cross validation on the scaled data.

In [ ]:
cv = StratifiedKFold(n_splits=10,shuffle=True, random_state=959)
model = LogisticRegression(solver='lbfgs', random_state=959, max_iter = 1e5)
stratified_cross_validate(model, X, y, cv=cv)

Scaling the data drastically improves the accuracy of our model. With scaled data the `Mean Accuracy` of our logistic regression model is 88.8%. While it takes a bit longer to fit than the unscaled model, the improvement in accuracy is worth the CV Time tradeoff. Let's run the basic parameters through a SearchGrid and see if that helps the model's performance.

### Logistic Regression Iteration 3 - GridSearch on Scaled Data

##### GridSearch to Identify the best parameters

Due to the number of combinations, we will use 3-fold cross validation so the CV time is not prohibitiely long."

In [ ]:
#Create a Logistic Regression object and perform a grid search to find the best parameters
#Adopted from https://github.com/jakemdrew/DataMiningNotebooks/blob/master/07.%20Regression.ipynb

logreg = LogisticRegression()
penalty = ['l2']
C = (1, 5, 10)
solver = ['sag', 'saga', 'lbfgs']

parameters = dict(C=C, penalty=penalty, solver=solver, max_iter = 1e5)

#Create a grid search object using the  
from sklearn.model_selection import GridSearchCV
logregGridSearch = GridSearchCV(estimator=logreg
                   , verbose=0 # low verbosity
                   , param_grid=parameters
                   , cv=3 # KFolds = 10 
                   , n_jobs = -1
                               )

#Perform hyperparameter search to find the best combination of parameters for our data
logregGridSearch.fit(X,y)

In [ ]:
#Adopted from https://www.kaggle.com/enespolat/grid-search-with-logistic-regression
#Print out our best parameters to plug into our logistic regression model

print("The optimal parameters according to the gridsearch are: " ,logregGridSearch.best_params_)



Now we'll re-run our logistic regression model after discovering the optimal parameters.

In [ ]:
cv = StratifiedKFold(n_splits=10,shuffle=True, random_state=959)
logreg_clf = LogisticRegression(penalty='l2', C=10, solver='lbfgs', random_state=959, max_iter = 1e5)
stratified_cross_validate(logreg_clf, X, y, cv=cv)

It seems that tuning the parameters based on the GridSearch increased our accuracy even closer to 100%. Similarly, it has decreaed the CV time form over 100 seconds to 42 seconds. We will use the tuned Logistic Regression iteration as our final classifier in the comparison.

### Regression Section 

In [ ]:
housing_data_regr = housing_data[(housing_data.SALEPRICECPY > 1e5) & (housing_data.SALEPRICECPY < 9.9e6)]

In [ ]:
housing_data_regr = housing_data_regr.drop(columns = ['SALEPRICE'], axis = 1)

In [ ]:
housing_data_regr = grouper_impute(dataframe_2 = housing_data_regr, grouper_col = 'ZIPCODE', grouper_impute = 'SALEPRICECPY', replace_val = 0, transfrmtn = 'median')

In [ ]:
feat_of_int = [ 'NEIGHBORHOOD', 'BUILDINGCLASSCATEGORY', 'ZIPCODE', 'DECADEBUILT','TAXCLASSATPRESENT', 'BUILDINGCLASSATPRESENT', 'TIMEOFMONTH','BUILDCLASSGENER']
housing_ml_df_regr = housing_data_regr[['NEIGHBORHOOD','BUILDINGCLASSCATEGORY','ZIPCODE', 'DECADEBUILT', 'TAXCLASSATPRESENT', 
                              'BUILDINGCLASSATPRESENT', 'TIMEOFMONTH','BUILDCLASSGENER', 
                              'RESIDENTIALUNITS', 'COMMERCIALUNITS', 'TOTALUNITS',
                              'LANDSQUAREFEET', 'GROSSSQUAREFEET','YEARBUILT',
                              'SALEPRICECPY','BUILDAGE','BOROUGH']]
ml_df_enc_regr = create_dummy_encod(ml_df = housing_ml_df_regr,features_of_interest = feat_of_int, drop_first_cat=True, sparsity=True)

In [ ]:
y_regr = ml_df_enc_regr['SALEPRICECPY'].values
X_regr = ml_df_enc_regr.drop(columns = ['SALEPRICECPY'], axis = 1).values

#### Capped Linear Regression

In [ ]:
#Create scaler object 
ML_std_scalr = StandardScaler()

In [ ]:
#Create a Linear Regression object and perform a grid search to find the best parameters
#Adopted from https://github.com/jakemdrew/DataMiningNotebooks/blob/master/07.%20Regression.ipynb
cv = ShuffleSplit(n_splits=10, test_size=0.20, random_state=959)

ML_std_scalr.fit(X_regr)

X_regr_scl = ML_std_scalr.transform(X_regr)


linreg = CappedLinearRegression()
parameters = {'normalize':(True,False), 'fit_intercept':(True,False)}

#Create a grid search object using the  
from sklearn.model_selection import GridSearchCV
regGridSearch = GridSearchCV(estimator=linreg
                   , verbose=1 # low verbosity
                   , param_grid=parameters
                   , cv=cv # KFolds = 10
                   , scoring=mae_scorer)

#Perform hyperparameter search to find the best combination of parameters for our data
regGridSearch.fit(X_regr_scl,y_regr)


In [ ]:
#Adopted from https://github.com/jakemdrew/DataMiningNotebooks/blob/master/07.%20Regression.ipynb

regGridSearch.best_estimator_

In [ ]:
#Create CappedLinearRegression predictions between 0 and 100% using the best parameters for our Linear Regression object
#Adopted from https://github.com/jakemdrew/DataMiningNotebooks/blob/master/07.%20Regression.ipynb

regEstimator = regGridSearch.best_estimator_

cv = ShuffleSplit(n_splits=10, test_size=0.20, random_state=959)

#Evaluate the regression estimator above using our pre-defined cross validation and scoring metrics. 
scoresResults_CappedReg =  EvaluateRegressionEstimator(regEstimator, X_regr_scl,y_regr, cv)

mae_avg_CappedLinearRegression = scoresResults_CappedReg['MAE'].mean()
scoresResults_CappedReg

#### Lasso Regression

In [ ]:
#Adopted from https://github.com/jakemdrew/DataMiningNotebooks/blob/master/07.%20Regression.ipynb

ML_std_scalr.fit(X_regr)

X_regr_scl = ML_std_scalr.transform(X_regr)

cv = ShuffleSplit(n_splits=10, test_size=0.20, random_state=959)
reg = Lasso(fit_intercept=True, normalize=True,copy_X=True
          , max_iter=10000, precompute=True, tol=0.0001, random_state=0)

#Test parameters 
alpha = [0.001, 0.1, 1, 10, 20]
selection = ['cyclic','random']
warm_start = [True, False]
parameters = {'alpha': alpha, 'selection': selection, 'warm_start': warm_start}

#Create a grid search object using the parameters above
from sklearn.model_selection import GridSearchCV
regGridSearch = GridSearchCV(estimator=reg
                   , n_jobs=8 # jobs to run in parallel
                   , verbose=1 # low verbosity
                   , param_grid=parameters
                   , cv=cv # KFolds = 10
                   , scoring=mae_scorer)

#Perform hyperparameter search to find the best combination of parameters for our data
regGridSearch.fit(X_regr_scl,y_regr)

In [ ]:
regGridSearch.best_estimator_

In [ ]:
#Adopted from https://github.com/jakemdrew/DataMiningNotebooks/blob/master/07.%20Regression.ipynb

#Create a regression estimator with best parameters for cross validation

cv = ShuffleSplit(n_splits=10, test_size=0.20, random_state=959)

regEstimator = Lasso(alpha=10, copy_X=True, fit_intercept=True, max_iter=10000, normalize=True,
      positive=False, precompute=True, random_state=0, selection='random',
      tol=0.0001, warm_start=True)
#Evaluate the regression estimator above using our pre-defined cross validation and scoring metrics.
scoresResults_Lasso = EvaluateRegressionEstimator(regEstimator, X_regr,y_regr, cv)

mae_avg_Lasso = scoresResults_Lasso['MAE'].mean()
scoresResults_Lasso

#### Gradient Boosting Regressor

In [ ]:

#Adopted from https://github.com/jakemdrew/DataMiningNotebooks/blob/master/07.%20Regression.ipynb

ML_std_scalr.fit(X_regr)


X_regr_scl = ML_std_scalr.transform(X_regr)

cv = ShuffleSplit(n_splits=10, test_size=0.20, random_state=959)
linreg = GradientBoostingRegressor()

parameters = { 'loss' : ['ls']
              ,'learning_rate' : [1e-3, 1e-1]
              ,'n_estimators': [500] 
              ,'criterion': ['mae']
              ,'min_samples_split':[2,3,4,5]
              ,'min_samples_leaf': [10, 25, 50]
              ,'max_features' : ['auto']
              ,'subsample' : [1e-2]
              ,'random_state': [0]
             }



              

#Create a grid search object using the  
from sklearn.model_selection import GridSearchCV
regGridSearch = GridSearchCV(estimator=linreg
                   , n_jobs=8 # jobs to run in parallel
                   , verbose=1 # low verbosity
                   , param_grid=parameters
                   , cv=cv # KFolds = 10
                   , scoring=mae_scorer)

#Perform hyperparameter search to find the best combination of parameters for our data
regGridSearch.fit(X_regr_scl , y_regr)

In [ ]:
#Adopted from https://github.com/jakemdrew/DataMiningNotebooks/blob/master/07.%20Regression.ipynb
#Display the best estimator parameters
regGridSearch.best_estimator_

In [ ]:
#Adopted from https://github.com/jakemdrew/DataMiningNotebooks/blob/master/07.%20Regression.ipynb

#Create a regression estimator with best parameters for cross validation
regEstimator = regGridSearch.best_estimator_

#Evaluate the regression estimator above using our pre-defined cross validation and scoring metrics.
scoresResults_GradientBoostingRegressor = EvaluateRegressionEstimator(regEstimator, X_regr_scl , y_regr, cv)


mae_avg_GradientBoostingRegressor = scoresResults_GradientBoostingRegressor['MAE'].mean()
scoresResults_GradientBoostingRegressor

# Modeling and Evaluation 4
>Analyze the results using your chosen method of evaluation. Use visualizations of the results to bolster the analysis. Explain any visuals and analyze why they are interesting to someone that might use this model.*

### Classification

##### Confusion matrix for KNN:

In [ ]:
#Confusion Matrix for K=1

cv = StratifiedKFold(n_splits=10,shuffle=True, random_state=959)

# Break our data into 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=959, stratify=y)

yhat = np.zeros(y.shape) # we will fill this with predictions

# get a handle to the classifier object, which defines the type
knn_clf = KNeighborsClassifier(n_neighbors=1, weights='uniform', metric='euclidean')

# now iterate through and get predictions, saved to the correct row in yhat
for train, test in cv.split(X,y):
    knn_clf.fit(X[train],y[train])
    yhat[test] = knn_clf.predict(X[test])


#confusion matrix for our KNN iteration 1
cm = mt.confusion_matrix(y, yhat)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.imshow(cm_normalized,cmap=plt.get_cmap('coolwarm'),aspect='auto')
plt.grid(False)

As we can see in the confusion matrix above, the model primarily misclassifies the target bouroughs as bourough 3 (2 on the confusion matrix).  This makes sense as observations for borough 3 were more than 40% of our dataset.  

##### Confustion Matrix for MultinomialNB

In [ ]:
#Reset Data for MinMaxScalar
y = ml_df_enc['BOROUGH'].values
X = ml_df_enc.drop(columns = ['BOROUGH'], axis = 1).values

#Scale Data
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Break our data into 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=959, stratify=y)

#confusion matrix for our Logistic Regression iteration 2
mnb_clf= MultinomialNB(alpha=0.001)
mnb_clf.fit(X_train, y_train)
y_pred = mnb_clf.predict(X_test)



# Create a confusion matrix to see what classes the logistic regression is getting wrong 


plt.subplots(figsize=(10, 10))
cm = confusion_matrix(y_true=y_test, y_pred=y_pred)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.imshow(cm_normalized,cmap=plt.get_cmap('coolwarm'),aspect='auto')
plt.xlabel('Predicted')
plt.ylabel('Expected')
plt.grid(False)

When we look at the confusion matrix for the MultinomialNB model, we can see that the misclassification issue with borough 3 is no longer present.  This makes sense as the imporovement in accuracy above the KNN model would have been improvements in the correct classificaiton of those boroughs.

##### Confusion Matrix for Logistic Regression 

In [ ]:
#Reset Data for StandardScalar
y = ml_df_enc['BOROUGH'].values
X = ml_df_enc.drop(columns = ['BOROUGH'], axis = 1).values

#scale data
X = ML_std_scalr.fit_transform(X)

# Break our data into 70% training and 30% test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=959, stratify=y)

#confusion matrix for our Logistic Regression iteration 2
lr_clf = LogisticRegression(penalty='l2', C=10, solver='lbfgs', random_state=959, max_iter = 1e5)
lr_clf.fit(X_train, y_train)
y_pred = lr_clf.predict(X_test)

# Create a confusion matrix to see what classes the logistic regression is getting wrong 

plt.subplots(figsize=(10, 10))
cm = confusion_matrix(y_true=y_test, y_pred=y_pred)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
plt.imshow(cm_normalized,cmap=plt.get_cmap('coolwarm'),aspect='auto')
plt.xlabel('Predicted')
plt.ylabel('Expected')
plt.grid(False)

What we can observe with the confusion matrix above is similar to the confusion matrix we saw with the MultinomialNB model.  This color scale can easily detect misclassifications (as well as show our school spirit), and it would follow that the confusion matrix appears pritine as the tuned logistic regression model provided a 10-fold cross-validated score of nearly 100%.  

### Regression

# Modeling and Evaluation 5
>*Discuss the advantages of each model for each classification task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical comparison methods. You must use statistical comparison techniques—be sure they are appropriate for your chosen method of validation as discussed in unit 7 of the course.*

### Classification

#### Statistical Analysis of KNN vs Naive Bayes

In [ ]:
# %load -r 1-15 statcompare.py
clf1 = knn_clf #Top performing KNN Model

#Reset Data for MinMaxScalar
y = ml_df_enc['BOROUGH'].values
X = ml_df_enc.drop(columns = ['BOROUGH'], axis = 1).values

#Scale Data
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
                         
clf2 = mnb_clf #Top performing Naive Bayes


from sklearn.model_selection import cross_val_score
# is clf1 better or worse than clf2?
cv = StratifiedKFold(n_splits=10,shuffle=True, random_state=959)
acc1 = cross_val_score(clf1, X, y=y, cv=cv)
acc2 = cross_val_score(clf2, X, y=y, cv=cv)

### Correction for multiple tests

Since we are doing two concurent hypothesis tests, we will do a Bonferroni correction by dividing alpha by 2 (0.5/2=0.025).  Our new critical t-value becomes 2.76
>T-value derived from: https://www.statisticshowto.com/tables/t-distribution-table/

In [ ]:
# %load -r 19-28 statcompare.py
t = 2.76 / np.sqrt(10)

e = (1-acc1)-(1-acc2)
# std1 = np.std(acc1)
# std2 = np.std(acc2)
stdtot = np.std(e)

dbar = np.mean(e)
print ('Range of:', dbar-t*stdtot,dbar+t*stdtot )
print (np.mean(acc1), np.mean(acc2))

h0: The mean accuracy of KNN and the mean acuracy of Multinomial Naive Bayes is the same
ha: The mean accuracies of the two models are different


###### Conclusion 1:
Given the confidence interval of the differnece does not include zero, we have sufficient evidence to reject h0.  We are 95% confident that the mean accuracy of MultinomialNB is between 10.39% and 17.95% higher than KNN.

Given that there is no statistical difference between the accuracies of Multinomial Naive Bayes and Logistic Regression, we can look at secondary metrics, such as runtime.  In our model classification iterations, Multinomial Naive Bayes had a much lower cross validation time. Niave Bayes took only 3-5 seconds verus over 100 seconds with Logistic Regression.  Although the speed of our model isn't necessarily instrumental for deployment, it makes sense to utilize a faster model with the same accuracy--Niave Bayes.  

### Statistical Comparison ov Naive Bayes vs Logistic Regression

In [ ]:
#Reset Data for MinMaxScalar
y = ml_df_enc['BOROUGH'].values
X = ml_df_enc.drop(columns = ['BOROUGH'], axis = 1).values

#Scale Data
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

clf1 = mnb_clf #Top performing Naive Bayes

In [ ]:
# %load -r 1-15 statcompare.py


#clf1 = mnb_clf #Top performing Naive Bayes

#Reset Data for StandardScalar to use Logistic Regression
y = ml_df_enc['BOROUGH'].values
X = ml_df_enc.drop(columns = ['BOROUGH'], axis = 1).values

#scale data
X = ML_std_scalr.fit_transform(X)

clf1 = lr_clf #Top performing logistic Regression model

#Reset Data for MinMaxScalar
y = ml_df_enc['BOROUGH'].values
X = ml_df_enc.drop(columns = ['BOROUGH'], axis = 1).values

#Scale Data
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
                         
clf2 = mnb_clf #Top performing Naive Bayes

from sklearn.model_selection import cross_val_score
# is clf1 better or worse than clf2?
cv = StratifiedKFold(n_splits=10,shuffle=True, random_state=959)
acc1 = cross_val_score(clf1, X, y=y, cv=cv)
acc2 = cross_val_score(clf2, X, y=y, cv=cv)

In [ ]:
# %load -r 19-28 statcompare.py
t = 2.76 / np.sqrt(10)

e = (1-acc1)-(1-acc2)
# std1 = np.std(acc1)
# std2 = np.std(acc2)
stdtot = np.std(e)

dbar = np.mean(e)
print ('Range of:', dbar-t*stdtot,dbar+t*stdtot )
print (np.mean(acc1), np.mean(acc2))

h0: The mean accuracy of Multinomial Naive Bayes and the mean acuracy of Logistic Regression is the same

ha: The mean accuracies of the two models are different

Given the confidence interval of the differnece includes zero, we do not have sufficient evidence to reject h0.  Therefore, we can not confidently conclude that Mulinomial Naive Bayes and Logistic Regression have different mean accuracies.

Given that there is no statistical difference between the accuracies of Multinomial Naive Bayes and Logistic Regression, we can look at secondary metrics, such as runtime.  In our model classification iterations, Multinomial Naive Bayes had a much lower cross validation time. Niave Bayes took only 3-5 seconds verus over 100 seconds with Logistic Regression.  Although the speed of our model isn't necessarily instrumental for deployment, it makes sense to utilize a faster model with the same accuracy--Niave Bayes.  

### Regression

In [ ]:
#Adapted from 
#https://github.com/jakemdrew/DataMiningNotebooks/blob/master/06.%20Classification.ipynb

#Gradient Bosting
regr1 = GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='mae', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features='auto', max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=10, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=500,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=0, subsample=0.01, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)



regest1 = Pipeline(
    [('scaler',ML_std_scalr),
     ('regr',regr1)]
)

cv = ShuffleSplit(n_splits=10, test_size=0.20, random_state=959)
reg_err1 = cross_val_score(regest1, X_regr, y_regr, cv=cv, scoring = 'neg_mean_absolute_error')


In [ ]:
#Lasso
regr2 = Lasso(alpha=10, copy_X=True, fit_intercept=True, max_iter=10000, normalize=True,
      positive=False, precompute=True, random_state=0, selection='random',
      tol=0.0001, warm_start=True)

regest2 = Pipeline(
    [('scaler',ML_std_scalr),
     ('regr',regr2)]
)

cv = ShuffleSplit(n_splits=10, test_size=0.20, random_state=959)

reg_err2 = cross_val_score(regest2, X_regr, y_regr, cv=cv, scoring = 'neg_mean_absolute_error')

In [ ]:
#Adapted from 
#https://github.com/jakemdrew/DataMiningNotebooks/blob/master/06.%20Classification.ipynb

#CappedLinear Regression
regr3 = CappedLinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                       normalize=False)

regest3 = Pipeline(
    [('scaler',ML_std_scalr),
     ('regr',regr3)]
)

cv = ShuffleSplit(n_splits=10, test_size=0.20, random_state=959)

reg_err3 = cross_val_score(regest3, X_regr, y_regr, cv=cv, scoring = 'neg_mean_absolute_error')

h0: The mean of MAE of the Gradient Boosting Regressor equals the mean of MAE of the LASSO regressor

ha: The mean MAE of the models are different

Comparing GradientBoostingRegressor and Lasso, the 95% confidence interval [43411,55324] shows that the interval doesn't includes zero, which implies that the two models are statistically different and we reject null hypothesis. There is evidence to suggest the LASSO model is statistically better at prediciting sale price. 

In [ ]:
# wanting to check 3 models, so bonferroni correction for alpha = 0.05 / 2 = 0.025therefore t = 2.76
# we reject h0 the models are statistically different, as zero is not in the interval, The LASSO model is the better 
#regressor model. 


t = 2.76 / np.sqrt(10)

e = abs(reg_err1) - abs(reg_err2) 

stdtot = np.std(e)

dbar = np.mean(e)
print ('Range of:', dbar-t*stdtot,dbar+t*stdtot )
print (abs(np.mean(reg_err1)), abs(np.mean(reg_err2)))

h0: The mean of MAE of the LASSO Regressor equals the mean of MAE of the Capped regressor

ha: The mean MAE of the models are different

Comparing Lasso and CappedLinearRegression, the 95% confidence interval [-10027,-946] shows that the interval doesn't includes zero, which implies that the two models are statistically different and we reject null hypothesis. There is sufficient evidence to sau the Lasso model performs better than the capped linear regression model. But one has to consider the practical significance, as there is roughly a ~50k, given the dynamic range of SALEPRICECPY the model is trying to predict, 1e5 to 9.9e6, 50k may or may not be signficant to the consumers of the model. It is something worth mentioning, especially if the capped linear regression model is eaiser to implement. 

In [ ]:
# fail to reject h0, the models are not statistically different, 
#there is not sufficient evidence to suggest the error from the capped linear regression and the lasso regression 
# is different. 
t = 2.76 / np.sqrt(10)

e =  abs(reg_err2) - abs(reg_err3)

stdtot = np.std(e)

dbar = np.mean(e)
print ('Range of:', dbar-t*stdtot,dbar+t*stdtot )
print (abs(np.mean(reg_err2)), abs(np.mean(reg_err3)))

# Modeling and Evaluation 6
>*Which attributes from your analysis are most important? Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classification task.*

### Classification

For the classification task, we will review the most important features in the Logistic Regression model. 

In [ ]:
#Reset Data for StandardScalar to use Logistic Regression
y = ml_df_enc['BOROUGH'].values
X = ml_df_enc.drop(columns = ['BOROUGH'], axis = 1).values

#scale data
X = ML_std_scalr.fit_transform(X)

clf = lr_clf #Top performing logistic Regression model

In [ ]:
clf = LogisticRegression(penalty='l2', C=10, solver='lbfgs', random_state=959)
clf.fit(X, y)

In [ ]:
#Creating a dictionary of the attributes and their coefficients 
coef_dict = {}
for coef, feat in zip(clf.coef_[0,:], ml_df_enc.drop(columns = ['BOROUGH'], axis = 1).columns):
    coef_dict[feat] = coef

In [ ]:
#code adopted from: https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value
coef_dict_sorted = {k: v for k, v in sorted(coef_dict.items(), key=lambda item: item[1])}


In [ ]:
plt.bar(coef_dict_sorted.keys(), coef_dict_sorted.values())


This chart is unreadable, so we will sort our dictionary to find the top ten highest attributes (the negative coefficients don't go below -0.1, so we can focus on positive).

In [ ]:
#https://stackoverflow.com/questions/7971618/python-return-first-n-keyvalue-pairs-from-dict
coef_top_ten = {k:coef_dict_sorted[k] for k in list(coef_dict_sorted)[-10:] }

In [ ]:
plt.bar(coef_top_ten.keys(), coef_top_ten.values())
plt.xticks(rotation=90)
plt.title('Top Ten Weighted Features')
plt.xlabel('Feature')
plt.ylabel('Weight')
plt.show()

In [ ]:
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.from_dict.html

#Table to view the exact values
pd.DataFrame.from_dict(coef_top_ten, orient='index', columns=["Weight"])

#### Interpretation of weights 

As we can see from isolating the ten attributes with the highest weights, they pertain to either neighborhoods or zip codes.  This isn't surprising, as there are distinct neighborhood names that occur in perhaps one borough.  This is the same with zip code.  We saw earlier that there is overlpa, but we assumed it would be a very helpful feature in classifying the correct borough for our applicaiton. 

Even though we have the top ten heighest-weighted featuers, let's take a look at what they mean relative to the reference class--Class 0, which is borough 1 (Manhattan).

##### NEIGHBORHOOD_GREENWICH VILLAGE-WEST:
>NEIGHBORHOOD_GREENWICH VILLAGE-WEST has a weight of 0.39. This is a categorical attribute, therefore we can interpret the weight as a change from the reference value of NEIGHBORHOOD_GREENWICH VILLAGE-WEST 0 to NEIGHBORHOOD_GREENWICH VILLAGE-WEST 1 (yes) increases the odds of the the borough being classified as 0 (Manhattan) by 48%, or a multiplicative factor of 1.48.

##### NEIGHBORHOOD_UPPER EAST SIDE:
>NEIGHBORHOOD_UPPER EAST SIDE has a weight of 0.40. This is a categorical attribute, therefore we can interpret the weight as a change from the reference value of NEIGHBORHOOD_UPPER EAST SIDE 0 to NEIGHBORHOOD_UPPER EAST SIDE 1 (yes) increases the odds of the the borough being classified as 0 (Manhattan) by 49%, or a multiplicative factor of 1.49.

    
##### NEIGHBORHOOD_HARLEM-UPPER:
>NEIGHBORHOOD_HARLEM-UPPERER has a weight of 0.40. This is a categorical attribute, therefore we can interpret the weight as a change from the reference value of NEIGHBORHOOD_HARLEM-UPPER 0 to NEIGHBORHOOD_HARLEM-UPPER 1 (yes) increases the odds of the the borough being classified as 0 (Manhattan) by 49%, or a multiplicative factor of 1.49.
    
##### NEIGHBORHOOD_UPPER EAST SIDE:
>NNEIGHBORHOOD_UPPER EAST SIDE has a weight of 0.43. This is a categorical attribute, therefore we can interpret the weight as a change from the reference value of NNEIGHBORHOOD_UPPER EAST SIDE 0 to NEIGHBORHOOD_UPPER EAST SIDE 1 (yes) increases the odds of the the borough being classified as 0 (Manhattan) by 54%, or a multiplicative factor of 1.54.
    
    
##### NEIGHBORHOOD_INWOOD:
>NNEIGHBORHOOD_UPPER EAST SIDE has a weight of 0.43. This is a categorical attribute, therefore we can interpret the weight as a change from the reference value of NNEIGHBORHOOD_UPPER EAST SIDE 0 to NEIGHBORHOOD_UPPER EAST SIDE 1 (yes) increases the odds of the the borough being classified as  0 (Manhattan) by 54%, or a multiplicative factor of 1.54.
    
##### ZIPCODE_10031:
>ZIPCODE_10031 has a weight of 0.45. This is a categorical attribute, therefore we can interpret the weight as a change from the reference value of ZIPCODE_10031 0 to ZIPCODE_10031 1 (yes) increases the odds of the the borough being classified as 0 (Manhattan) by 57%, or a multiplicative factor of 1.57.
    
##### ZIPCODE_10029:
>ZIPCODE_10029 has a weight of 0.48. This is a categorical attribute, therefore we can interpret the weight as a change from the reference value of ZIPCODE_10029 0 to ZIPCODE_10029 1 (yes) increases the odds of the the borough being classified as 0 (Manhattan) by 62%, or a multiplicative factor of 1.62.
    
##### NEIGHBORHOOD_HARLEM-EAST:
>NEIGHBORHOOD_HARLEM-EAST has a weight of 0.54. This is a categorical attribute, therefore we can interpret the weight as a change from the reference value of NEIGHBORHOOD_HARLEM-EAST 0 to NEIGHBORHOOD_HARLEM-EAST 1 (yes) increases the odds of the the borough being classified as 0 (Manhattan) by 72%, or a multiplicative factor of 1.72.
    
##### NEIGHBORHOOD_CHELSEA:
>NEIGHBORHOOD_CHELSEA has a weight of 0.56. This is a categorical attribute, therefore we can interpret the weight as a change from the reference value of NEIGHBORHOOD_CHELSEA 0 to NEIGHBORHOOD_CHELSEA 1 (yes) increases the odds of the the borough being classified as 0 (Manhattan) by 75%, or a multiplicative factor of 1.75.
    
##### NEIGHBORHOOD_HARLEM-CENTRAL:
>NEIGHBORHOOD_HARLEM-CENTRAL has a weight of 0.65. This is a categorical attribute, therefore we can interpret the weight as a change from the reference value of NEIGHBORHOOD_HARLEM-CENTRAL 0 to NEIGHBORHOOD_HARLEM-CENTRAL 1 (yes) increases the odds of the the borough being classified as 0 (Manhattan) by 92%, or a multiplicative factor of 1.92.

### Regression

In [ ]:
#https://scikit-learn.org/stable/auto_examples/feature_selection/plot_rfe_with_cross_validation.html#sphx-glr-auto-examples-feature-selection-plot-rfe-with-cross-validation-py
#https://nbviewer.jupyter.org/github/jakemdrew/EducationDataNC/blob/master/2017/Models/2017SegregatedElementarySchoolCampuses.ipynb
#Create a regression estimator with best parameters for cross validation

cv = ShuffleSplit(n_splits=10, test_size=0.20, random_state=959)

regrlasso = Lasso(alpha=10, copy_X=True, fit_intercept=True, max_iter=10000, normalize=True,
      positive=False, precompute=True, random_state=0, selection='random',
      tol=0.0001, warm_start=True)
 

pipe = Pipeline([('scaler',ML_std_scalr)]) 

pipe.fit(X_regr, y_regr)

X_regr_scl_p = pipe.transform(X_regr)

rfecv = RFECV(estimator=regrlasso, step=1, cv=cv, scoring='neg_mean_absolute_error')
rfecv.fit(X_regr_scl_p, y_regr)

In [ ]:
%matplotlib inline

print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (LASSO NEG MSE)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

In [ ]:
plt.style.use('ggplot')

rfe_ft_imp_df = pd.DataFrame({'feature_names':ml_df_feats.columns, 'weights':rfecv.grid_scores_})
rfe_ft_imp_df.sort_values(by='weights', inplace=True, ascending=False )

top50features = rfe_ft_imp_df.head(50)

top50features

# Deployment
>*How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.?*


This model will be very useful for buying or selling real estate in NY. It provides price prediction and trends in each Borough.

The measurement of the model value is number of users using the model on daily basis. Usage logs will be captured and analyzed constantly to assess model value and appropriate actions are taken to retain the customers.

After training and testing the final model we will make it available for the interested parties or customers by deploying it to production environment. One way to do this is to save the trained Scikit-Learn model using joblib, including the full preprocessing and prediction pipeline into production environment.

Option1: This strategy is by wrapping the model with a dedicated REST API end point hosted on any Web server which servers HTTP/HTTPS protocols. This pattern will insulate the interface and easier to upgrade the model with new version.

Option2: As various cloud platforms are offering AI/ML hosting platforms. Like Google Cloud Platform (GCP) AI Platform. We will save best model using joblib and upload it to Google Cloud Storage (GCS), then on the Google Clod AI Platform create a new model version, pointing to the GCS file. This will provide load balanced and scaled REST web service.

Both options take JSON requests as input data and returns JSON responses containing the predictions. The RESTful API can be consumed by any third-party applications which can consume REST API’s. In addition, we will deploy web application which provides UI to interact with the API’s mentioned above. where users can consume as SaaS. Any software which goes to production need to be monitored to check model’s live performance and availability. Logs need to be captured for monitoring. Note: It is not always possible to determine the model’s performance without human analysis. Human intervention is also required to maintain the health of the deployed model.

We will refresh the data and train the model every one week, which keep up with seasonal price variation trends by Borough.

# Exceptional Work
>*You have free reign to provide additional analyses. One idea: grid search parameters in a parallelized fashion and visualize the performances across attributes. Which parameters are most significant for making a good model for each classification algorithm?*

Please award accordinly to using grid search techniques on the SALEPRICECPY regression models, and for using a Gradient Boosting Regressor for the regression of SALEPRICECPY with the grid search and the grid search for the logistic regression. 

Also, please see the below SMOTE analysis for balancing multi-classification for BOROUGH. 

# SMOTE ANALYSIS for Multi-Classification for BOROUGH

One interesting observation we observed for the classification task occured using SMOTE sampling from the imblearn package. We observed mean accuracy of 100% in the case of scaled SMOTE data using the helper CV test function and .99% using unscaled SMOTE data. Even more interesting, we found by just scaling the data, the accuracy was similar to using Scaled SMOTE sampling. 

This prompted us to explore whether there had been bleed over from the CV sets into the test sets. 

The following analysis will review a file of hashed values for the features to explore what is really occuring in the CV folds and then perform a 3 CV experiment on SMOTE data, SMOTE Scaled Data and Scaled Original Data using the same logistic regression and the same 3 CV data sets. 



We see the original housing data for the BOROUGHS multi level classification problem is largely imbalanced, with class 3 having the largest number of occurances. 

In [ ]:
y_num_class = housing_ml_df.BOROUGH.values
plot_class_dist(y_num_class) 

We see the SMOTE function has upsampled the data to balance all of the classes for the target value. 

In [ ]:
#https://imbalanced-learn.readthedocs.io/en/stable/over_sampling.html#smote-variants

y = ml_df_enc['BOROUGH'].values
X = ml_df_enc.drop(columns = ['BOROUGH'], axis = 1).values

smte = SMOTE(sampling_strategy='not majority')
X_smote, y_smote = smte.fit_sample(X, y)

plot_class_dist(y_smote)

Let us review the Hashed values of our original encoded data set and perform SMOTE upsampling to see what the algorthim is accomplishing. 

In [ ]:
ml_df_enc_hashed_url = 'https://raw.githubusercontent.com/andrewmejia600/MSDS7331/andrew/RAW_DATA/ml_df_enc_cpy_hash.csv'

ml_df_enc_hashed = pd.read_csv(ml_df_enc_hashed_url)

ml_df_enc_hashed.info()


In [ ]:
ml_df_enc_hashed.hash_key

In [ ]:
ml_df_enc_hashed.columns

We see the classes are remain balanced. 

In [ ]:
y_hash = ml_df_enc_hashed['BOROUGH'].values

X_hash = ml_df_enc_hashed.drop(columns = ['BOROUGH', 'Unnamed: 0'], axis = 1).values

smte = SMOTE(sampling_strategy='not majority')
X_smote_hash, y_smote_hash = smte.fit_sample(X_hash, y_hash)

plot_class_dist(y_smote_hash)

Using our Hashed encoded data, lets perform a 10K stratified split, to ensure we still have even balance.

In [ ]:
sskf = StratifiedKFold(n_splits=10,shuffle=True, random_state=959)
sskf.get_n_splits(X_smote_hash, y_smote_hash)

test_idx = {}
train_idx = {}
counter = 1
for train_index, test_index in sskf.split(X_smote_hash, y_smote_hash):
    
    k,v = counter, X_smote_hash[train_index][:,-1]
    train_idx.update({k : v})
    
    k,v = counter, X_smote_hash[test_index][:,-1]
    test_idx.update({k : v})
    
    counter+=1
    

We will now collect all of the hashes from the CV splits into dictionaries for both the Test Folds and the Training Folds. 

In [ ]:
#Adapted from 
#https://stackoverflow.com/questions/19736080/creating-dataframe-from-a-dictionary-where-entries-have-different-lengths
test_folds = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in test_idx.items() ]))
test_folds 

In [ ]:
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.melt.html#pandas.DataFrame.melt
test_folds_melt = test_folds.copy()
test_folds_melt['idx'] = test_folds_melt.index
test_folds_melt = test_folds_melt.melt(id_vars=['idx'], value_vars=[1,2,3,4,5,6,7,8,9,10], var_name ='Fold')
test_folds_melt.rename(columns = {'Idx' : 'OrigIdx', 'value' : 'SMOTEIdx'}, inplace = True) 
test_folds_melt

In [ ]:
test_folds_melt = test_folds_melt.dropna(axis = 0)
test_folds_melt

In [ ]:
test_folds_melt['SMOTEHASH'] = test_folds_melt.SMOTEIdx
test_folds_melt = test_folds_melt.drop(columns = 'SMOTEIdx')

In [ ]:
test_folds_melt.head(n=20)

We see there are not any duplicate hashes within the Test Folds, meaning, a hash did not repeat internally in its own fold or externally into another fold. 

In [ ]:
test_folds_melt[test_folds_melt.duplicated(['SMOTEHASH'], keep=False)]

Now lets review the train folds. 

In [ ]:
train_folds = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in train_idx.items() ]))
train_folds

In [ ]:
train_folds_melt = train_folds.copy()
train_folds_melt['idx'] = train_folds_melt.index
train_folds_melt = train_folds_melt.melt(id_vars=['idx'], value_vars=[1,2,3,4,5,6,7,8,9,10], var_name = 'Fold')
train_folds_melt.rename(columns = {'Idx' : 'OrigIdx', 'value' : 'SMOTEIdx'}, inplace = True) 
train_folds_melt

In [ ]:
train_folds_melt = train_folds_melt.dropna(axis = 0)
train_folds_melt

In [ ]:
train_folds_melt['SMOTEHASH'] = train_folds_melt.SMOTEIdx
train_folds_melt = train_folds_melt.drop(columns = 'SMOTEIdx')

In [ ]:
train_folds_melt

We see in the train folds, there are duplicated hashes, this is to be expected with CV. 

In [ ]:
train_folds_melt[train_folds_melt.duplicated(['SMOTEHASH'], keep=False)]

We see the train folds are making 9 train folds and 1 test fold for our CV split. Again this is expected. 

In [ ]:
train_folds_melt.groupby(['SMOTEHASH']).count()

Let's turn our attention to how many SMOTE hashes are found in both the training folds and the test folds. We see that there are considerable intersections. 

In [ ]:
all_smote_idxs = train_folds_melt.merge(test_folds_melt, how = 'inner', on='SMOTEHASH', suffixes = ['Train', 'Test'])

In [ ]:
all_smote_idxs.shape

Now, lets see how many are duplicated on the hash. 

In [ ]:
all_smote_idxs_d = all_smote_idxs[all_smote_idxs.duplicated(['SMOTEHASH'], keep=False)].reset_index().drop(columns = ['idxTrain', 'idxTest', 'index'], axis = 1)

In [ ]:
all_smote_idxs_d

We can see while the Training fold has a duplicate hash, each test fold is being held out and not making into its own fold for training. 

In [ ]:
all_smote_idxs_d[all_smote_idxs_d.duplicated(['SMOTEHASH', 'FoldTest'], keep=False)].head(n=50)

Now, let us see how this impacts our models. 

We will first try a method as perscribed by the referenced article https://beckernick.github.io/oversampling-modeling/ of how to time the SMOTE technique. 

The article essentially states one should only apply SMOTE to the train data set, and then test on a hold out set. 

We call when we optimized the parameters and used a Stratified 10 K fold on a SMOTE data set where we did not control the SMOTE applying to only the train data set, we got an accuarcy score of 1. This seemed highly suspicious. 

We will use 5 fold Stratified CV for the interest of time. 

For reference, we see on the original data, the overall accuracy from the 5 fold split is not better than .50. 

In [ ]:
y = ml_df_enc['BOROUGH'].values
X = ml_df_enc.drop(columns = ['BOROUGH'], axis = 1).values


cv = StratifiedKFold(n_splits=5,shuffle=True, random_state=959)
lr_clf = LogisticRegression(solver = 'lbfgs', multi_class='multinomial', n_jobs=-1, random_state=959)



for train_index, test_index in cv.split(X, y): 
    
    lr_clf.fit(X[train_index],y[train_index])

    print(lr_clf.score(X[test_index], y[test_index])) 

Using pure SMOTE acually decreased the accuracy accross the folds. 

In [ ]:
#Adapted from 
#https://beckernick.github.io/oversampling-modeling/

y = ml_df_enc['BOROUGH'].values
X = ml_df_enc.drop(columns = ['BOROUGH'], axis = 1).values

cv = StratifiedKFold(n_splits=5,shuffle=True, random_state=959)
lr_clf = LogisticRegression(solver = 'lbfgs', multi_class='multinomial', n_jobs=-1, random_state=959)

for train_index, test_index in cv.split(X, y): 
    X_smote_train, y_smote_train = smte.fit_sample(X[train_index], y[train_index])
    lr_clf.fit(X_smote_train,y_smote_train)
    print(lr_clf.score(X[test_index], y[test_index])) 

Let's apply SMOTE to scaled training Data only and test against scaled test data. 

In [ ]:
y = ml_df_enc['BOROUGH'].values
X = ml_df_enc.drop(columns = ['BOROUGH'], axis = 1).values

#Create scaler object 
ML_std_scalr = StandardScaler()

cv = StratifiedKFold(n_splits=5,shuffle=True, random_state=959)
lr_clf = LogisticRegression(solver = 'lbfgs', multi_class='multinomial', n_jobs=-1, random_state=959)



for train_index, test_index in cv.split(X, y): 
    ML_std_scalr.fit(X)
    X_smote_scl = ML_std_scalr.transform(X)
    X_smote_train, y_smote_train = smte.fit_sample(X_smote_scl[train_index], y[train_index])
    lr_clf.fit(X_smote_train,y_smote_train)
    print(lr_clf.score(X_smote_scl[test_index], y[test_index])) 

Let's see what happens when we merely scale the data and do not apply SMOTE. 

In [ ]:
y = ml_df_enc['BOROUGH'].values
X = ml_df_enc.drop(columns = ['BOROUGH'], axis = 1).values


cv = StratifiedKFold(n_splits=5,shuffle=True, random_state=959)
lr_clf = LogisticRegression(solver = 'lbfgs', multi_class='multinomial', n_jobs=-1, random_state=959)



for train_index, test_index in cv.split(X, y): 
    ML_std_scalr.fit(X)
    X_scl = ML_std_scalr.transform(X)
    lr_clf.fit(X_scl[train_index],y[train_index])

    print(lr_clf.score(X_scl[test_index], y[test_index])) 

# Conclusion



We see that the indices are repeated in the test sets when using stratified k fold and SMOTE sampling. While there is not bleed over within the test folds, we do see bleed over from the train folds into the test folds, but the test folds do not appear to be overlapping. We can say the SMOTE technique from imblearn does work as advertised. 

We see that controlling for when we conduct SMOTE sampling and test on a hold out set, we do not necessarily see a gain in accuracy performance on this data set. 

Perhaps what is even more interesting is when we tested SMOTE scaled data v.s. the original data after being scaled. We see the results are identical and SMOTE is not even required for this data set. 

### ADDITIONAL REGRESSION MODELS FOR EXCEPTIONAL POINTS

In [ ]:
y_regr = ml_df_enc['SALEPRICE'].values
X_regr = ml_df_enc.drop(columns = ['SALEPRICE'], axis = 1).values

#https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html#sklearn.linear_model.SGDRegressor
#https://scikit-learn.org/stable/modules/generated/sklearn.kernel_approximation.Nystroem.html#sklearn.kernel_approximation.Nystroem
cv = ShuffleSplit(n_splits=10, test_size=0.20, random_state=959)

reg = SGDRegressor()

feature_map_nystroem = Nystroem(gamma=.2,
                                random_state=1,
                                n_components=400)

X_regr_transformed = feature_map_nystroem.fit_transform(X_regr)

#Set up SVR parameters to test (WARNING: Creates 80 models!!!) 
loss = ['squared_loss', 'epsilon_insensitive']
alphas = [0.0001, 0.1]
penalty = ['l2']
parameters = {'loss': loss, 'alpha' : alphas, 'penalty': penalty }

#Create a grid search object using the parameters above
from sklearn.model_selection import GridSearchCV
regGridSearch = GridSearchCV(estimator=reg
                   , n_jobs=8 # jobs to run in parallel
                   , verbose=1 # low verbosity
                   , param_grid=parameters
                   , cv=cv # KFolds = 10
                   , scoring=mae_scorer)

#Perform hyperparameter search to find the best combination of parameters for our data
regGridSearch.fit(X_regr_transformed,y_regr)


In [ ]:
regGridSearch.best_estimator_

In [ ]:
regEstimator = regGridSearch.best_estimator_

#Evaluate the regression estimator above using our pre-defined cross validation and scoring metrics. 
EvaluateRegressionEstimator(regEstimator, X_regr_transformed ,y_regr, cv)